In [1]:
from standard_function import *
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_excel("ICD_new.xlsx", header = None, encoding = 'utf-8')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10126 entries, 0 to 10125
Data columns (total 3 columns):
0    10126 non-null object
1    10126 non-null object
2    10126 non-null object
dtypes: object(3)
memory usage: 237.4+ KB


In [3]:
df1=''
for i in range(0, df.shape[0]):
    df1 += df[0][i][0:3]+ " "

In [4]:
def Convert(string): 
    li = list(string.split(" ")) 
    return li 
       
df2 = Convert(df1)

In [5]:
df[3] = pd.Series(df2)

In [6]:
data_frame = df[[0,2,3]]
data_frame.rename(index=int, columns={0: 0, 2: 1, 3: 2 })
data_frame = data_frame.drop([0])

In [7]:
target = data_frame[3]
data_frame[2] = data_frame[2].apply(delete_special_char)
#data_frame[2] = data_frame[2].apply(un_punctuation)

In [8]:
from nltk import ngrams

def kieu_ngram(string, n=1):
    gram_str = list(ngrams(string.split(), n))
    return [ " ".join(gram).lower() for gram in gram_str ]

data_frame["3gram"] = data_frame[2].apply(lambda t: kieu_ngram(t, 3))
#data_frame["2gram"] = data_frame[2].apply(lambda t: kieu_ngram(t, 2))


In [9]:
train_data = data_frame["3gram"].tolist()
train_data

[['bệnh tả do',
  'tả do vibrio',
  'do vibrio cholerae',
  'vibrio cholerae 01',
  'cholerae 01 typ',
  '01 typ sinh',
  'typ sinh học',
  'sinh học cholerae'],
 ['bệnh tả do',
  'tả do vibrio',
  'do vibrio cholerae',
  'vibrio cholerae 01',
  'cholerae 01 typ',
  '01 typ sinh',
  'typ sinh học',
  'sinh học eltor'],
 ['bệnh tả không', 'tả không đặc', 'không đặc hiệu'],
 ['bệnh thương hàn',
  'thương hàn và',
  'hàn và phó',
  'và phó thương',
  'phó thương hàn'],
 [],
 ['bệnh phó thương', 'phó thương hàn', 'thương hàn a'],
 ['bệnh phó thương', 'phó thương hàn', 'thương hàn b'],
 ['bệnh phó thương', 'phó thương hàn', 'thương hàn c'],
 ['bệnh phó thương',
  'phó thương hàn',
  'thương hàn không',
  'hàn không đặc',
  'không đặc hiệu'],
 ['nhiễm salmonella khác'],
 ['viêm ruột do', 'ruột do salmonella'],
 ['nhiễm trùng huyết', 'trùng huyết do', 'huyết do salmonella'],
 ['nhiễm salmonella khu', 'salmonella khu trú'],
 ['nhiễm trùng salmonella',
  'trùng salmonella xác',
  'salmonella xá

In [10]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile
import logging

path = get_tmpfile("3_gram.model")
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)    
model = Word2Vec(train_data, size=100, window=10, min_count=1, workers=4, sg=0)
model.save("3_gram.model")

print(" Saved Model !")

2019-04-27 11:43:21,025 : INFO : collecting all words and their counts
2019-04-27 11:43:21,026 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-04-27 11:43:21,040 : INFO : PROGRESS: at sentence #10000, processed 61773 words, keeping 23925 word types
2019-04-27 11:43:21,041 : INFO : collected 24306 word types from a corpus of 62837 raw words and 10125 sentences
2019-04-27 11:43:21,041 : INFO : Loading a fresh vocabulary
2019-04-27 11:43:21,068 : INFO : effective_min_count=1 retains 24306 unique words (100% of original 24306, drops 0)
2019-04-27 11:43:21,068 : INFO : effective_min_count=1 leaves 62837 word corpus (100% of original 62837, drops 0)
2019-04-27 11:43:21,119 : INFO : deleting the raw counts dictionary of 24306 items
2019-04-27 11:43:21,120 : INFO : sample=0.001 downsamples 11 most-common words
2019-04-27 11:43:21,121 : INFO : downsampling leaves estimated 61810 word corpus (98.4% of prior 62837)
2019-04-27 11:43:21,152 : INFO : estimated require

 Saved Model !


In [43]:
model.wv.similar_by_word('nhiễm trùng khu')

[('bệnh giun đũa', 0.4537196159362793),
 ('thừng tinh màng', 0.4295884370803833),
 ('cốt hóa khác', 0.33972203731536865),
 ('dạ giả trong', 0.33919650316238403),
 ('tử cung chưa', 0.33346065878868103),
 ('thuật nội ngoại', 0.3286118507385254),
 ('nhiều cơ và', 0.3174431025981903),
 ('nướu hàm trên', 0.3164563775062561),
 ('chậu nữ do', 0.3154296875),
 ('chất của nhau', 0.3153294026851654)]